<center>
    <h2> Time Dependent Density Functional Theory with QEpy</h2>
    <h2>Tutorial</h2>
</center>
<table>
  <tr>
      <td><p><h3>Rutgers University-Newark</h3></p><p>Dr Xuecheng Shao</p><p>Dr Kaili Jiang (Microsoft)</p><p>Dr. Xin Chen</p><p>Jessica Martinez</p><p>Valeria Rios </p><p>Alina Umerbekova</p><p>Nicholas Viot</p></td>
          <td><img src="../figures/logos/logo.jpg" width=800 /></td>
  </tr>
</table>

Quantum Multiscale School June 2024

## Goal
#### Obtain the Optical Spectra of Bezene using QEpy


To run this tutorial we need to import QEpy and some of its modules

In [ ]:
import qepy
from qepy.driver import Driver
from qepy.io import QEInput

Download PP

In [ ]:
additional_files = {
    'H.pz-vbc.UPF' : 'https://pseudopotentials.quantum-espresso.org/upf_files/H.pz-vbc.UPF',
    'C.pz-vbc.UPF' : 'https://pseudopotentials.quantum-espresso.org/upf_files/C.pz-vbc.UPF',
}
from dftpy.formats import download_files
download_files(additional_files)

Generate the qe_options for the SCF calculation

In [ ]:
scf_options = {
    '&control': {'calculation': "'scf'",
                 'outdir': "'tmp'",
                 'prefix': "'c6h6'",
                 'pseudo_dir': "'./'",
                 'restart_mode': "'from_scratch'",
                 'tprnfor': True,
                 'tstress': False},
    '&electrons': {'conv_thr': 1e-10,
                   'diagonalization': "'david'"},
    '&system': {'celldm(1)': 24.45306579840016,
                'celldm(2)': 1.0,
                'celldm(3)': 0.8,
                'ecutwfc': 20,
                'ibrav': 8,
                'nat': 12,
                'nosym': True,
                'ntyp': 2},
    'atomic_positions angstrom': ['H   3.97999999999988   5.00000000000000   3.50000000000000',
                                  'C   5.07999999999987   5.00000000000000   3.50000000000000',
                                  'C   5.77500000000000   6.20377531126040   3.50000000000000',
                                  'H   5.22500000000007   7.15640325542330   3.50000000000000',
                                  'C   5.77500000000000   3.79622468873960   3.50000000000000',
                                  'H   5.22500000000007   2.84359674457670   3.50000000000000',
                                  'C   7.16500000000000   6.20377531126040   3.50000000000000',
                                  'H   7.71499999999993   7.15640325542330   3.50000000000000',
                                  'C   7.16500000000000   3.79622468873960   3.50000000000000',
                                  'H   7.71499999999993   2.84359674457670   3.50000000000000',
                                  'C   7.86000000000013   5.00000000000000   3.50000000000000',
                                  'H   8.96000000000012   5.00000000000000   3.50000000000000'],
    'atomic_species': ['C    12.00000  C.pz-vbc.UPF', 'H     1.00000  H.pz-vbc.UPF'],
    'k_points automatic': ['1 1 1    0 0 0'],
}

Generate the qe_options for the TDDFT calculation

In [ ]:
tddft_options = {
    '&inputtddft': {'dt': 2.0,
                    'e_direction': 1,
                    'job': "'optical'",
                    'l_tddft_restart': False,
                    'nstep': 5000,
                    'prefix': "'c6h6'",
                    'tmp_dir': "'tmp/'"},
}

Write the input files

In [ ]:
# Write the options to input files for comparison with traditional command way
scf_in = 'C6H6.scf.in'
tddfpt_in= 'C6H6.tddft.in'
QEInput().write_qe_input(scf_in, qe_options=scf_options, prog='pw')
QEInput().write_qe_input(tddfpt_in, qe_options=tddft_options, prog='cetddft')

In [ ]:
driver = Driver(scf_in, task='scf', logfile='tmp.scf.out')

In [ ]:
driver.scf()

In [ ]:
driver.save()

Create a driver that computes the time evolution of the system to obtain the optical spectra

In [ ]:
driver = Driver(tddfpt_in, task='optical', logfile='tmp.tddft.out', iterative=True)

Perform a TDDFT calculation with 100 steps (Runner)

In [ ]:
max_steps = 100
for istep in range(max_steps):
    driver.diagonalize() # driver.propagate()
    print("\r", end="")
    print(f"Progress: [{istep+1}/{max_steps}]", "|" * (istep*50 // max_steps), end="", flush=True)

Perform a TDDFT calculation with 500 steps (Runner)

In [ ]:
max_steps = 500
for istep in range(max_steps):
    driver.diagonalize()
    print("\r", end="")
    print(f"Progress: [{istep+1}/{max_steps}]", "|" * (istep*50 // max_steps), end="", flush=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Import <b>calc_spectra_mu</b> from DFTpy (damping function and FFT)

In [ ]:
from dftpy.td.utils import calc_spectra_mu

Set up important constants

In [ ]:
import qepy
from qepy.qepy_modules import constants
au_sec = constants.get_au_sec()
as2au = 1.0E-18 / au_sec
bohr = constants.get_angstrom_au()
ha = constants.get_autoev()

Define the direction and the strength of the kick and read the output from the TDDFT calculation

In [ ]:
direction = 0 # 0, 1, 2 means x, y, z-direction, respectively
k = 0.01*bohr # kick_strength in a.u.
interval = 2.0 * as2au
output = 'tmp.tddft.out'
# output = 'C6H6.tddft.out'

In [ ]:
mu = []
with open(output, 'r') as fh:
    for line in fh:
        if line[:4]=='DIP ':
            mu.append(list(map(float, line.split()[-3:])))
mu = np.asarray(mu)

In [ ]:
max_steps = len(mu)-1
t = np.linspace(0, interval * max_steps, max_steps + 1)
delta_mu = mu[:,0] - mu[0,0]
plt.plot(t, delta_mu)
plt.xlabel('Time (au)')
plt.ylabel('Dipole Moment (au)')

In [ ]:
omega, spectra = calc_spectra_mu(delta_mu, interval, kick=k, emax=2, sigma=0.02*(as2au**2))
plt.plot(omega*ha, spectra, '-')
plt.xlabel('Frequency (eV)')
plt.ylabel('Intensity (au)')
plt.xlim(0, 40)

# Challenge

<b>Compute the optical spectra of Ethylene C2H4</b>

Important aspect

Create another folder

Hint use ASE to build the molecule get the scaled positions of the molecule to build the scf_options